# Deploy and Run the SQD IEF-PCM Function Template

### *A project in collaboration with the Cleveland Clinic Foundation*

This interactive guide shows how to upload the SQD IEF-PCM function template to Qiskit Serverless and run an example workload.

### Requirements

This guide was last run with the following package versions:

In [ ]:
from qiskit_ibm_catalog import __version__ as catalog_version
from qiskit_ibm_runtime import __version__ as runtime_version

print("qiskit-ibm-catalog: ", catalog_version)
print("qiskit-ibm-runtime: ", runtime_version)

# Simulated molecular system


In this tutorial we show how to calculate the ground state energy and solvation free energy of methanol molecule in implicit solvent using the SQD IEF-PCM method. Here the methanol molecule is the solute, the electron structure of which is simulated explicitly, and the solvent is water, approximated as a continuous dielectric medium. To account for the [electron correlation effects](https://onlinelibrary.wiley.com/doi/epdf/10.1002/ijch.202100111) in methanol, while maintaining the balance between the computational cost and accuracy, we only include the $\sigma$, $\sigma^{*}$, and lone pair orbitals in the active space simulated with SQD IEF-PCM. This orbital selection is done with [atomic valence active space (AVAS) method](https://github.com/pyscf/pyscf.github.io/blob/master/examples/mcscf/43-avas.py) using the C[2s,2p], O[2s,2p], and H[1s] atomic orbital components, which results in the active space of 14 electrons and 12 orbitals (14e,12o). The reference orbitals are calculated with closed-shell Hartree Fock using the cc-pvdz basis set.

# LUCJ options

The initial step of SQD method is the execution of LUCJ quantum circuit, which sample a set of computational basis states from the probability distribution of the molecular system. To achieve the balance between the depth of the LUCJ circuit and it is expressibility the qubits corresponding to the spin orbitals with the opposite spin have the two-qubit gates applied between them only in instances when these qubits are neighboring each other through single ancilla qubit. To implement this approach on IBM hardware with a heavy-hex topology, qubits representing the spin orbitals with the same spin are connected through a line topology where each line take a zig-zag shape due the heavy-hex connectivity of the target hardware, while the qubits representing the spin orbitals with the opposite spin only have connection at every 4th qubit.

The user has to provide the `initial_layout` array corresponding to the qubits that satisfy this [_zig-zag_ pattern](https://pubs.rsc.org/en/content/articlehtml/2023/sc/d3sc02516k) in the `lucj_options` section of the SQD IEF-PCM function. In case of SQD IEF-PCM (14e,12o)/cc-pvdz simulations of methanol we chose the initial qubit layout corresponding to the main diagonal of the Eagle R3 QPU. Here the first 12 elements of the `initial_layout` array `[0, 14, 18, 19, 20, 33, 39, 40, 41, 53, 60, 61, ...]` correspond to the alpha spin orbitals and the last 12 elements `[... 2, 3, 4, 15, 22, 23, 24, 34, 43, 44, 45, 54]` correspond to beta spin orbitals.

Importantly, the user has to make a decision regarding the `number_of_shots` which corresponds to number of measurements in LUCJ circuit. The number of shots needs to be suefficiently large due to the fact that the first step of S-CORE procedure relies on the samples in the right particle sector to obtain the initial approximation to the ground-state occupation number distribution. 

The number of shots is highly system- and hardware-dependent, but [noncovalent](https://arxiv.org/abs/2410.09209), [fragment-based](https://arxiv.org/abs/2411.09861), and [implicit solvent](https://pubs.acs.org/doi/10.1021/acs.jpcb.5c01030) SQD studies sugget that one can reach the chemical accuracy using:

- 20,000 - 200,000 shots for systems with less than 16 molecular orbitals (32 spin orbitals)
- 200,000 shots for systems with 16 - 18 molecular orbitals
- 200,000 - 2,000,000 shots for systems with more than 18 molecular orbitals

The required number of shots will be affected not only by the number of spin orbitals in the studied system, but also by the size of the Hilbert space corresponding to the selected active space within the studied system. Generally instances with smaller Hilbert spaces will require fewer shots. Other LUCJ options available to the user are [circuit transpiler optimization level](https://docs.quantum.ibm.com/guides/set-optimization) and [error suppresion options](https://docs.quantum.ibm.com/guides/configure-error-suppression), where these options are also going to affect the required number of shots and the resulting accuracy.

# SQD options
Most critical options in SQD simulations are the `sqd_iterations`, `number_of_batches`, and `samples_per_batch`. Generally, the lower number of samples per batch can be counteracted with more batches (`number_of_batches`) and more iterations of S-CORE (`sqd_iterations`). 
- More batches allow user to sample more variations of the configurational subspaces. Since the lowest-energy batch is taken as the solution for the ground state energy of the system, higher number of batches can improve the results through better statistics. 
- Additional iterations of S-CORE allow to recover more configurations from original LUCJ distribution if the number of samples in correct particle sector is low. This allows user to reduce the number of samples per batch.

Alternative strategy is to use the higher number of samples per batch, which ensures that most of the initial LUCJ samples in right particle space are used during the S-CORE procedure and individual subspaces encapsulate suefficient variety of electron configurations. In turn, this reduces the number of required S-CORE steps, where only 2 or 3 iterations of SQD are needed if number of samples per batch is large enough. However, more samples per batch results in higher computational cost of each diagonalization step. Hence, the balance between the accuracy and computational cost in SQD simulations can be achieved through optimal choice of `sqd_iterations`, `number_of_batches`, and `samples_per_batch` options.

The [SQD IEF-PCM study](https://pubs.acs.org/doi/10.1021/acs.jpcb.5c01030) shows that when 3 iterations of S-CORE are used the chemical accuracy can be reached with:

- 600 samples per batch in methanol SQD IEF-PCM (14e,12o) simulations
- 1500 samples per batch in methylamine SQD IEF-PCM (14e,13o) simulations
- 6000 samples per batch in water SQD IEF-PCM (8e,23o) simulations
- 16000 samples per batch in ethanol SQD IEF-PCM (20e,18o) simulations

Just like the required number of shots in LUCJ, the required number of samples per batch used in S-CORE procedure is highly system- and hardware-dependent. The examples above can be used to estimate the initial point for the benchmark of required number of samples per batch. The tutorial on systematic benchmark of required number of sample per batch can be found [here](https://qiskit.github.io/qiskit-addon-sqd/how_tos/choose_subspace_dimension.html).

## Steps in the execution of SQD IEF-PCM function

## 1. Authentication

Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API key (token), which you can find on the [IBM Quantum Platform](https://quantum.cloud.ibm.com) dashboard. This will allow you to locally instantiate the serverless client to upload or run the selected function:

```python
from qiskit_ibm_catalog import QiskitServerless
serverless = QiskitServerless(token="MY_TOKEN")
```

You can optionally use `save_account()` to save your credentials in your local environment (see the [Set up your IBM Cloud account](/docs/guides/cloud-setup#cloud-save) guide). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#save_account):

```python
QiskitServerless.save_account(token="MY_TOKEN")
```

If the account is saved, there is no need to provide the token to authenticate:

In [ ]:
from qiskit_ibm_catalog import QiskitServerless

# Authenticate to the remote cluster.
# In this case, using the "ibm_quantum_platform" (IBM Cloud) channel
serverless = QiskitServerless(channel="ibm_quantum_platform")

## 2. Upload the custom function

To upload a custom Qiskit Function, you must first instantiate a `QiskitFunction` object that defines the function source code. The title will allow you to identify the function once it's in the remote cluster. The main entry point is the file that contains `if __name__ == "__main__"`. If your workflow requires additional source files, you can define a working directory that will be uploaded together with the entry point.

In [ ]:
from qiskit_ibm_catalog import QiskitFunction

template = QiskitFunction(
    title="sqd_pcm_template",
    entrypoint="sqd_pcm_entrypoint.py",
    working_dir="./source_files/",  # all files in this directory will be uploaded
)
print(template)

Once the instance is ready, upload it to serverless:

In [ ]:
serverless.upload(template)

To check if the program successfully uploaded, use `serverless.list()`:

In [ ]:
serverless.list()

## 3. Load and run the custom function remotely


The function template has been uploaded, so you can run it remotely with Qiskit Serverless. First, load the template by name:

In [ ]:
template = serverless.load("sqd_pcm_template")
print(template)

Next, run the template with the domain-level inputs for SQD-IEF PCM. This example specifies a methanol-based workload.

In [ ]:
molecule = {
    "atom": """
    O -0.04559 -0.75076 -0.00000;
    C -0.04844 0.65398 -0.00000;
    H 0.85330 -1.05128 -0.00000;
    H -1.08779 0.98076 -0.00000;
    H 0.44171 1.06337 0.88811;
    H 0.44171 1.06337 -0.88811
    """,  # Must be specified
    "basis": "cc-pvdz",  # default is "sto-3g"
    "spin": 0,  # default is 0
    "charge": 0,  # default is 0
    "verbosity": 0,  # default is 0
    "number_of_active_orb": 12,  # Must be specified
    "number_of_active_alpha_elec": 7,  # Must be specified
    "number_of_active_beta_elec": 7,  # Must be specified
    "avas_selection": ["%d O %s" % (k, x) for k in [0] for x in ["2s", "2px", "2py", "2pz"]]
    + ["%d C %s" % (k, x) for k in [1] for x in ["2s", "2px", "2py", "2pz"]]
    + ["%d H 1s" % k for k in [2, 3, 4, 5]],  # default is None
}

solvent_options = {
    "method": "IEF-PCM",  # other available methods are COSMO, C-PCM, SS(V)PE, see https://manual.q-chem.com/5.4/topic_pcm-em.html
    "eps": 78.3553,  # value for water
}

lucj_options = {
    "initial_layout": [
        0,
        14,
        18,
        19,
        20,
        33,
        39,
        40,
        41,
        53,
        60,
        61,
        2,
        3,
        4,
        15,
        22,
        23,
        24,
        34,
        43,
        44,
        45,
        54,
    ],
    "dynamical_decoupling_choice": True,
    "twirling_choice": True,
    "number_of_shots": 200000,
    "optimization_level": 2,
}

sqd_options = {
    "sqd_iterations": 3,
    "number_of_batches": 10,
    "samples_per_batch": 1000,
    "max_davidson_cycles": 200,
}

backend_name = "ibm_fez"

In [ ]:
job = template.run(
    backend_name=backend_name,
    molecule=molecule,
    solvent_options=solvent_options,
    lucj_options=lucj_options,
    sqd_options=sqd_options,
)
print(job.job_id)

Check the detailed status of the job:

In [ ]:
import time

t0 = time.time()
status = job.status()
if status == "QUEUED":
    print(f"time = {time.time()-t0:.2f}, status = QUEUED")
while True:
    status = job.status()
    if status == "QUEUED":
        continue
    print(f"time = {time.time()-t0:.2f}, status = {status}")
    if status == "DONE" or status == "ERROR":
        break

After the job is running, you can fetch logs created from the `logger.info` outputs. These can provide actionable information about the progress of the SQD IEF-PCM workflow. For example, the same spin orbital connections, or the two-qubit depth of the final ISA circuit intended for execution on hardware.

In [ ]:
print(job.logs())

Calling for the job result blocks the rest of the program until a result is available. After the job is done, you can retrieve the results. These include the solvation free energy, as well as information the lowest energy batch, lowest energy value, and other useful information such as the total solver duration.

In [ ]:
result = job.result()

result

Note that the result metadata includes a resource usage summary that allows to better estimate the QPU and CPU time required for each workload (this example ran on a dummy device so actual resource usage times might differ).  

After the job completes, the entire logging output will be available.

In [ ]:
print(job.logs())